## Source:  https://tylerbanksblog.wordpress.com/2017/03/23/machine-learning-applied-to-fmri-data-in-python/

In [1]:
import scipy.io as sio
import random
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neural_network import BernoulliRBM
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn import preprocessing

In [39]:
fmri_contents = sio.loadmat('data-starplus-04820-v7.mat')

In [40]:
numberOfRois = 25
rois = ['CALC', 'LIPL', 'LT', 'LTRIA', 'LOPER', 'LIPS', 'LDLPFC']
vox_ind = [fmri_contents["meta"][0]["rois"][0]["columns"][0][j][0] for j in range(numberOfRois-1) if fmri_contents["meta"][0]["rois"][0]["name"][0][j][0] in rois]
vox = []
for i in range(len(vox_ind)):
    for j in range(len(vox_ind[i])):
        vox.append(vox_ind[i][j])

In [41]:
stop = 8;
frames = [i for i in range(stop)]

In [42]:
numberOfTrials = 54
data = []
classification = []
exclude = [0,26,27,32,37,46,51,53]#no data here
for ki, k in enumerate(frames):
    data.append([])
    classification.append([])
    offset_i = 0
    for i in range(numberOfTrials):
        if(i in exclude):
            offset_i += 1
            continue
        data[ki].append([])
        classification[ki].append([])
        offset_j = 0
        for ji, j in enumerate(vox):
            if(len(fmri_contents["data"][i][0]) == numberOfTrials+1):
                data[ki][i-offset_i].append([])
                data[ki][i-offset_i][ji-offset_j] = fmri_contents["data"][i][0][k][j-1]
            else:
                offset+=1 #if we didn't add an element
            classification[ki][i-offset_i].append(1 if fmri_contents["info"][0][i]["firstStimulus"][0] == "P" else 0)

In [43]:
len(data[7][45])

1888

In [44]:
frame = 7
numTrainingTrials = 36
c = list(zip(data[frame], classification[frame]))
random.shuffle(c)
training_data, training_classification = zip(*c[:numTrainingTrials])
testing_data, testing_classification = zip(*c[numTrainingTrials:])
clf = MLPClassifier(solver='sgd', alpha=1e-5 ,hidden_layer_sizes=(20,6), random_state=1, max_iter=1000)
clf.fit(training_data, np.array(training_classification).reshape(-1))
predict = np.array(clf.predict(testing_data))
actual = np.array(testing_classification).reshape(-1)
error = np.mean(predict != actual)
print ('Error     : ', error)
print ('prediction: ', predict)
print ('actual    : ', actual)
print ('f1Score&nbsp;&nbsp; : ', f1_score(actual, predict, average='weighted'))

ValueError: Found input variables with inconsistent numbers of samples: [36, 67968]